In [ ]:
from abc import ABC, abstractmethod
from pyspark.sql import SparkSession, DataFrame

# Inicializa a SparkSession (idealmente fora do módulo em produção)
spark = SparkSession.builder.appName("Data Export via Factory").getOrCreate()

# Interface de Exportação
class DataExport(ABC):
    @abstractmethod
    def export(self, df: DataFrame, path: str, destination: str = "local") -> None:
        """Exporta um DataFrame para o caminho especificado, com base no destino."""
        pass

# Implementação concreta para exportar em formato Parquet para o sistema de arquivos local
class ParquetExport(DataExport):
    def export(self, df: DataFrame, path: str, destination: str = "local") -> None:
        """Exporta o DataFrame para o sistema de arquivos local ou S3, dependendo do destino."""
        if destination == "local":
            self.export_local(df, path)
        elif destination == "s3":
            self.export_s3(df, path)
        else:
            raise ValueError("Destino inválido. Use 'local' ou 's3'.")

    def export_local(self, df: DataFrame, path: str) -> None:
        """Exporta o DataFrame para o sistema de arquivos local em formato Parquet."""
        df.write.mode("overwrite").parquet(path)

    def export_s3(self, df: DataFrame, path: str) -> None:
        """Exporta o DataFrame para o S3 em formato Parquet."""
        df.write.mode("overwrite").parquet(f"s3a://{path}")